# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import pprint

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,2.18,96,60,0.73,GS,1699738390
1,1,golfito,8.6390,-83.1660,26.27,93,100,0.92,CR,1699738882
2,2,zheleznogorsk-ilimskiy,56.5768,104.1217,-14.24,95,17,1.34,RU,1699738882
3,3,vorkuta,67.5000,64.0000,-6.61,97,100,3.83,RU,1699738882
4,4,miranda,3.2499,-76.2281,30.07,84,85,0.91,CO,1699738883


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    width = 1400,
    height = 850,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .7
)
# Display the map plot
hvplot.save(map_plot,"output_data/map_plot.html")

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
filtered_cities = city_data_df[(city_data_df['Max Temp'] > 21) \
                              & (city_data_df['Max Temp'] < 27) \
                              & (city_data_df['Wind Speed'] < 4.5) \
                              & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
filtered_cities = filtered_cities.dropna()

# Display sample data
filtered_cities.sample(12)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
346,346,awjilah,29.1081,21.2869,21.01,44,0,2.71,LY,1699738515
440,440,howrah,22.5892,88.3103,22.00,88,0,1.54,IN,1699738796
480,480,siwa oasis,29.2041,25.5195,21.35,50,0,0.64,EG,1699738969
494,494,guerrero negro,27.9769,-114.0611,24.79,23,0,4.30,MX,1699738570
306,306,mar del plata,-38.0023,-57.5575,25.01,77,0,3.60,AR,1699738936
101,101,sehwan,26.4279,67.8630,21.81,22,0,0.96,PK,1699738898
465,465,ukiah,39.1502,-123.2078,22.47,58,0,0.45,US,1699738967
24,24,laguna,38.4210,-121.4238,23.08,36,0,2.57,US,1699738853
466,466,aoulef,26.9667,1.0833,24.15,22,0,4.44,DZ,1699738967
55,55,blythe,33.6103,-114.5964,26.25,11,0,3.09,US,1699738605


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_cities[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.sample(12)

,City,Country,Lat,Lng,Humidity,Hotel Name
421,rodolfo sanchez taboada,MX,31.7167,-116.5667,58,
101,sehwan,PK,26.4279,67.8630,22,
306,mar del plata,AR,-38.0023,-57.5575,77,
145,campo formoso,BR,-10.5075,-40.3214,34,
440,howrah,IN,22.5892,88.3103,88,
465,ukiah,US,39.1502,-123.2078,58,
494,guerrero negro,MX,27.9769,-114.0611,23,
55,blythe,US,33.6103,-114.5964,11,
480,siwa oasis,EG,29.2041,25.5195,50,
24,laguna,US,38.4210,-121.4238,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [11]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "apiKey":geoapify_key, 
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row['Lng']
    latitude = row['Lat'] 
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
   
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
         #If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.sample(12)

Starting hotel search
laguna - nearest hotel: Holiday Inn Express & Suites
blythe - nearest hotel: Quality Inn
sehwan - nearest hotel: Dum Dastagir Resthouse
campo formoso - nearest hotel: Hotel Rio das Pedras
mar del plata - nearest hotel: Nuevo Ostende
awjilah - nearest hotel: No hotel found
rodolfo sanchez taboada - nearest hotel: Estero Beach Resort
howrah - nearest hotel: Sun India Guest House
ukiah - nearest hotel: Palace Hotel
aoulef - nearest hotel: دار الضياف
siwa oasis - nearest hotel: فندق الكيلانى
guerrero negro - nearest hotel: Plaza sal paraiso


,City,Country,Lat,Lng,Humidity,Hotel Name
440,howrah,IN,22.5892,88.3103,88,Sun India Guest House
24,laguna,US,38.4210,-121.4238,36,Holiday Inn Express & Suites
145,campo formoso,BR,-10.5075,-40.3214,34,Hotel Rio das Pedras
466,aoulef,DZ,26.9667,1.0833,22,دار الضياف
346,awjilah,LY,29.1081,21.2869,44,No hotel found
494,guerrero negro,MX,27.9769,-114.0611,23,Plaza sal paraiso
55,blythe,US,33.6103,-114.5964,11,Quality Inn
421,rodolfo sanchez taboada,MX,31.7167,-116.5667,58,Estero Beach Resort
480,siwa oasis,EG,29.2041,25.5195,50,فندق الكيلانى
306,mar del plata,AR,-38.0023,-57.5575,77,Nuevo Ostende


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display
map_plot2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    width = 1400,
    height = 850,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = .7,
    hover_cols=['Lng','Lat','City','Humidity','Hotel Name','Country']
)

# Display the map plot
hvplot.save(map_plot2,"output_data/Hotel_plot.html")